# Extracting data from DataFrames

In [2]:
import pandas as pd

## Slicing rows

The simplest way to slice rows of a DataFrame is to use the `.head()` or `.tail()` methods if the rows that you want are at the start or end of the DataFrame. 

To slice using labels, need to use the `.loc()` method. To slice columns, we need to specify both indices, with "all rows" (`:`) selected as the first index.

Recall that slicing with labels is inclusive of last label selected.

In [3]:
url = 'https://github.com/HeardLibrary/digital-scholarship/raw/master/data/r/wv5_women_and_development.csv'
development = pd.read_excel(url)
development.head()

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'\xef\xbb\xbfcount'

In [ ]:
# Create a slice (view) with only the fractions
state_co2_fuel_fractions = state_co2_fuel.loc[:, 'Coal fraction': 'Natural Gas fraction']
state_co2_fuel_fractions.head()

To slice rows, only the first index needs to be specified. For integer indices, use the `.iloc()` method.

In [ ]:
# Create a slice with only the top four states
top_state_co2_fuel = state_co2_fuel.iloc[:4]
# Note that included rows are 0, 1, 2, and 3 (but not 4).
top_state_co2_fuel

Combine both slicing operations at once.

In [ ]:
top_state_co2_fuel_fraction = state_co2_fuel.iloc[:4].loc[:, 'Coal fraction': 'Natural Gas fraction']
top_state_co2_fuel_fraction

# Selecting data

Units are million metric tons

In [ ]:
url = 'https://github.com/HeardLibrary/digital-scholarship/raw/master/data/codegraf/co2_data.xlsx'
state_co2 = pd.read_excel(url)
state_co2.head(15)

Performing a boolean operation on a column generates a series of booleans whose index matches the DataFrame rows

In [ ]:
state_co2.State=='Alabama'

The boolean series can be used to filter a subset of rows in the DataFrame.

Notice that the indices for the rows carry through in the selection.

In [ ]:
state_co2[state_co2.State=='Alaska']

In [ ]:
state_co2[state_co2['Sector']=='Industrial'].head()

You can assign the selection to a named view (but remember that indices are maintained).

In [ ]:
state_co2_industrial = state_co2[state_co2['Sector']=='Industrial']
state_co2_industrial.head()

## Dealing with missing data



In [ ]:
url = 'https://github.com/HeardLibrary/digital-scholarship/raw/master/data/gis/wg/Metro_Nashville_Schools.csv'
schools = pd.read_csv(url)
schools.head()

In some cases, cells were empty because the group wasn't represented (i.e. there were zero students). In that case, those `NaN` values should be zeros.

The first argument of the `.fillna()` method can be a single value if it applys to the entire table, or a dictionary if it applies only to certain columns.

In [ ]:
schools.fillna({'Native Hawaiian or Other Pacific Islander': 0}, inplace=True)
schools.head()

In other cases, cells were empty because that column didn't apply to that kind of school (e.g. high schools don't have PreK students). The `.dropna()` method can be used to skip rows with any `NaN` values, but that won't work if you only care about certain columns. In that case, we can filter rows using the `.notnull()` method. The `.isnull()` method can be used to select only rows that have `NaN` valued for a column.

In [ ]:
schools[schools['Grade PreK 3yrs'].notnull()]